# Part III: Model Evaluation Using NeMo Evaluator

This notebook covers the following:

0. [Pre-requisites: Configurations and Health Checks](#step-0)
1. [Establish a baseline accuracy benchmark](#step-1). This uses the off-the-shelf llama-3.2-1b-instruct model
2. [Evaluate the LoRA customized model](#step-2)

In [1]:
from time import sleep, time
from nemo_microservices import NeMoMicroservices

---
<a id="step-0"></a>
## Prerequisites: Configurations and Health Checks

Before you proceed, make sure that you completed the previous notebooks on data preparation and model fine-tuning to obtain the assets required to follow along.

### Configure NeMo Microservices Endpoints

The following code imports necessary configurations and prints the endpoints for the NeMo Data Store, Entity Store, Customizer, Evaluator, and NIM, as well as the namespace and base model.

In [2]:
from config import *

# # Initialize NeMo Microservices SDK client
# nemo_client = NeMoMicroservices(
#     base_url=NEMO_URL,
#     inference_base_url=NIM_URL,
# )

# Separate SDK clients per service
entity_client = NeMoMicroservices(base_url=NEMO_ENTITY_STORE_URL, inference_base_url=NIM_URL)
customizer_client = NeMoMicroservices(base_url=NEMO_CUSTOMIZER_URL, inference_base_url=NIM_URL)
evaluator_client = NeMoMicroservices(base_url=NEMO_EVALUATOR_URL, inference_base_url=NIM_URL)
guardrails_client = NeMoMicroservices(base_url=NEMO_GUARDRAILS_URL, inference_base_url=NIM_URL)

In [3]:
print(f"Data Store endpoint: {NEMO_DATA_STORE_URL}")
print(f"Entity Store endpoint: {NEMO_ENTITY_STORE_URL}")
print(f"Customizer endpoint: {NEMO_CUSTOMIZER_URL}")
print(f"Evaluator endpoint: {NEMO_EVALUATOR_URL}")
print(f"NIM endpoint: {NIM_URL}")
print(f"Namespace: {NMS_NAMESPACE}")
print(f"Base Model for Customization: {BASE_MODEL}@{BASE_MODEL_VERSION}")

Data Store endpoint: http://nemo-data-store.nemo.svc.cluster.local:3000
Entity Store endpoint: http://nemo-entity-store.nemo.svc.cluster.local:8000
Customizer endpoint: http://nemo-customizer.nemo.svc.cluster.local:8000
Evaluator endpoint: http://nemo-evaluator.nemo.svc.cluster.local:7331
NIM endpoint: http://nemo-nim-proxy.nemo.svc.cluster.local:8000
Namespace: xlam-tutorial-ns
Base Model for Customization: meta/llama-3.2-1b-instruct@v1.0.0+L40


### Check Available Models

Specify the customized model name that you got from the previous notebook to the following variable. 

In [4]:
CUSTOMIZED_MODEL = "xlam-tutorial-ns/llama-3.2-1b-xlam-run1@v1" # CUSTOM_MODEL # paste from the previous notebook

The following code checks if the NIM endpoint hosts the model properly.

In [5]:
# Check if the custom LoRA model is hosted by NVIDIA NIM
models = entity_client.inference.models.list()
model_names = [model.id for model in models.data]

assert CUSTOMIZED_MODEL in model_names, \
    f"Model {CUSTOMIZED_MODEL} not found"

HTTP Request: GET http://nemo-nim-proxy.nemo.svc.cluster.local:8000/v1/models "HTTP/1.1 200 OK"


### Verify the Availability of the Datasets

In the previous notebook, we uploaded the test dataset along with the train and validation sets using `nemo_client.datasets.create(name=DATASET_NAME, namespace=NMS_NAMESPACE, ...)`.
The following code performs a sanity check to validate the dataset by retrieving it and printing the URL of the files in the dataset.

In [6]:
# Sanity check to validate dataset
dataset = entity_client.datasets.retrieve(namespace=NMS_NAMESPACE, dataset_name=DATASET_NAME)
print("Files URL:", dataset.files_url)

HTTP Request: GET http://nemo-entity-store.nemo.svc.cluster.local:8000/v1/datasets/xlam-tutorial-ns/xlam-ft-dataset "HTTP/1.1 200 OK"


Files URL: hf://datasets/xlam-tutorial-ns/xlam-ft-dataset


---
<a id="step-1"></a>
## Step 1: Establish Baseline Accuracy Benchmark

First, we’ll assess the accuracy of the 'off-the-shelf' base model—pristine, untouched, and blissfully unaware of the transformative magic that is fine-tuning. 

### 1.1: Create an Evaluation Config Object
Create an evaluation configuration object for NeMo Evaluator. For more information on various parameters, refer to the [NeMo Evaluator configuration](https://docs.nvidia.com/nemo/microservices/latest/evaluate/evaluation-configs.html) in the NeMo microservices documentation.


* The `tasks.custom-tool-calling.dataset.files_url` is used to indicate which test file to use. Note that it's required to upload this to the NeMo Data Store and register with Entity store before using.
* The `tasks.dataset.limit` argument below specifies how big a subset of test data to run the evaluation on
* The evaluation metric `tasks.metrics.tool-calling-accuracy` reports `function_name_accuracy` and `function_name_and_args_accuracy` numbers, which are as their names imply.

In [7]:
simple_tool_calling_eval_config = {
    "type": "custom",
    "tasks": {
        "custom-tool-calling": {
            "type": "chat-completion",
            "dataset": {
                "files_url": f"hf://datasets/{NMS_NAMESPACE}/{DATASET_NAME}/testing/xlam-test-single.jsonl",
                "limit": 50
            },
            "params": {
                "template": {
                    "messages": "{{ item.messages | tojson}}",
                    "tools": "{{ item.tools | tojson }}",
                    "tool_choice": "auto"
                }
            },
            "metrics": {
                "tool-calling-accuracy": {
                    "type": "tool-calling",
                    "params": {"tool_calls_ground_truth": "{{ item.tool_calls | tojson }}"}
                }
            }
        }
    }
}

### 1.2: Launch Evaluation Job 

The following code calls the `nemo_client.evaluation.jobs.create()` method to launch an evaluation job in the NeMo Evaluator.
It uses the evaluation configuration defined in the previous cell and targets the base model.

In [8]:
# Create evaluation job for the base model
eval_job = evaluator_client.evaluation.jobs.create(
    config=simple_tool_calling_eval_config,
    target={"type": "model", "model": BASE_MODEL}
)

base_eval_job_id = eval_job.id
print(f"Created evaluation job: {base_eval_job_id}")
eval_job

HTTP Request: POST http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs "HTTP/1.1 201 Created"


Created evaluation job: eval-6EiFmVb4ed3X3TCKKXKSBd


EvaluationJob(config=EvaluationConfig(type='custom', id='eval-config-4UBwcuem1iswexdGBcxkcp', created_at=datetime.datetime(2025, 8, 25, 3, 47, 17, 244169), custom_fields={}, description=None, groups=None, name='eval-config-4UBwcuem1iswexdGBcxkcp', namespace='default', ownership=None, params=None, project=None, schema_version='1.0', tasks={'custom-tool-calling': TaskConfig(type='chat-completion', dataset=DatasetEv(files_url='hf://datasets/xlam-tutorial-ns/xlam-ft-dataset/testing/xlam-test-single.jsonl', id='dataset-Gn47mZTKb6EWSy5i5nV1WH', created_at=datetime.datetime(2025, 8, 25, 3, 47, 17, 244344), custom_fields={}, description=None, format=None, hf_endpoint=None, limit=50, name='dataset-Gn47mZTKb6EWSy5i5nV1WH', namespace='default', ownership=None, project=None, schema_version='1.0', split=None, type_prefix=None, updated_at=datetime.datetime(2025, 8, 25, 3, 47, 17, 244345), version_id='main', version_tags=[]), metrics={'tool-calling-accuracy': MetricConfig(type='tool-calling', params=

The following code defines a helper function to poll on job status until it finishes:

In [9]:
def wait_eval_job(nemo_client, job_id: str, polling_interval: int = 10, timeout: int = 6000):
    """Helper for waiting an eval job."""
    start_time = time()
    job = nemo_client.evaluation.jobs.retrieve(job_id=job_id)
    status = job.status

    while (status in ["pending", "created", "running"]):
        # Check for timeout
        if time() - start_time > timeout:
            raise RuntimeError(f"Took more than {timeout} seconds.")

        # Sleep before polling again
        sleep(polling_interval)

        # Fetch updated status and progress
        job = nemo_client.evaluation.jobs.retrieve(job_id=job_id)
        status = job.status

        # Progress details (only fetch if status is "running")
        progress = 0
        if status == "running" and job.status_details:
            progress = job.status_details.progress or 0
        elif status == "completed":
            progress = 100

        print(f"Job status: {status} after {time() - start_time:.2f} seconds. Progress: {progress}%")

    return job

Run the helper function:

In [11]:
# Poll
job = wait_eval_job(evaluator_client, base_eval_job_id, polling_interval=5, timeout=600)

HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-6EiFmVb4ed3X3TCKKXKSBd "HTTP/1.1 200 OK"
HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-6EiFmVb4ed3X3TCKKXKSBd "HTTP/1.1 200 OK"


Job status: running after 5.02 seconds. Progress: 94.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-6EiFmVb4ed3X3TCKKXKSBd "HTTP/1.1 200 OK"


Job status: running after 10.03 seconds. Progress: 96.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-6EiFmVb4ed3X3TCKKXKSBd "HTTP/1.1 200 OK"


...

Job status: running after 20.04 seconds. Progress: 98.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-6EiFmVb4ed3X3TCKKXKSBd "HTTP/1.1 200 OK"


Job status: running after 25.05 seconds. Progress: 100.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-6EiFmVb4ed3X3TCKKXKSBd "HTTP/1.1 200 OK"


Job status: completed after 30.06 seconds. Progress: 100%


### 1.3 Review Evaluation Metrics

The following code retrieves the evaluation results for the base evaluation job.

In [12]:
results = evaluator_client.evaluation.jobs.results(job_id=base_eval_job_id)
print(results.model_dump_json(indent=2, exclude_unset=True))

HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-6EiFmVb4ed3X3TCKKXKSBd/results "HTTP/1.1 200 OK"


{
  "job": "eval-6EiFmVb4ed3X3TCKKXKSBd",
  "id": "evaluation_result-XXLd1qjUdSTRd641fi5yKG",
  "created_at": "2025-08-25T03:47:17.425116",
  "custom_fields": {},
  "files_url": "hf://datasets/evaluation-results/eval-6EiFmVb4ed3X3TCKKXKSBd",
  "groups": {},
  "namespace": "default",
  "tasks": {
    "custom-tool-calling": {
      "metrics": {
        "tool-calling-accuracy": {
          "scores": {
            "function_name_accuracy": {
              "value": 0.34,
              "stats": {
                "count": 50,
                "mean": 0.34,
                "sum": 17.0
              }
            },
            "function_name_and_args_accuracy": {
              "value": 0.2,
              "stats": {
                "count": 50,
                "mean": 0.2,
                "sum": 10.0
              }
            }
          }
        }
      }
    }
  },
  "updated_at": "2025-08-25T04:01:22.037924"
}


The following code extracts and prints the accuracy scores for the base model.

In [13]:
# Extract function name accuracy score
base_scores = results.tasks["custom-tool-calling"].metrics["tool-calling-accuracy"].scores
base_function_name_accuracy_score = base_scores["function_name_accuracy"].value
base_function_name_and_args_accuracy = base_scores["function_name_and_args_accuracy"].value

print(f"Base model: function_name_accuracy: {base_function_name_accuracy_score}")
print(f"Base model: function_name_and_args_accuracy: {base_function_name_and_args_accuracy}")

Base model: function_name_accuracy: 0.34
Base model: function_name_and_args_accuracy: 0.2


Without any finetuning, the `meta/llama-3.2-1b-instruct` model should score in the ballpark of about 12% in `function_name_accuracy`, and 8% in `function_name_and_args_accuracy` (note that scores will vary by about +/-4% due to non-determinism of LLMs).

### (Optional) 1.4 Download and Inspect Results

To take a deeper look into the model's generated outputs, you can download and review the results.

In [ ]:
def download_evaluation_results(nemo_client, eval_job_id, output_file):
    """Downloads evaluation results for a given job ID."""
    
    try:
        # Get download results
        results = nemo_client.evaluation.jobs.download_results(job_id=eval_job_id)
        
        # Save the results to a file
        results.write_to_file(output_file)

        print(f"Evaluation results for job {eval_job_id} downloaded successfully to {output_file}.")
        return True
    except Exception as e:
        print(f"Failed to download evaluation results: {e}")
        return False

In [ ]:
output_file = f"{base_eval_job_id}.zip"

# Assertion fails if download fails
assert download_evaluation_results(nemo_client=nemo_client, eval_job_id=base_eval_job_id, output_file=output_file) == True

You can inspect the downloaded results file to observe places where the base model errors. Without any fine-tuning, some models not only return inaccurate function names and arguments, but they may not adhere to a consistent structured / predictable output schema. This makes it difficult to automatically parse these outputs, deterring integration with external systems.

---
<a id="step-2"></a>
## Step 2: Evaluate the LoRA Customized Model

### 2.1 Launch Evaluation Job

Run another evaluation job with the same evaluation config but with the customized model.

In [14]:
# Create evaluation job for customized model
ft_eval_job = evaluator_client.evaluation.jobs.create(
    config=simple_tool_calling_eval_config,
    target={"type": "model", "model": CUSTOMIZED_MODEL}
)

ft_eval_job_id = ft_eval_job.id
print(f"Created evaluation job for customized model: {ft_eval_job_id}")
ft_eval_job

HTTP Request: POST http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs "HTTP/1.1 201 Created"


Created evaluation job for customized model: eval-RFZY5WxsHQ4S6BMQmYb2aA


EvaluationJob(config=EvaluationConfig(type='custom', id='eval-config-C9gggwDWfUypZdtofEbwAE', created_at=datetime.datetime(2025, 8, 25, 4, 2, 29, 683769), custom_fields={}, description=None, groups=None, name='eval-config-C9gggwDWfUypZdtofEbwAE', namespace='default', ownership=None, params=None, project=None, schema_version='1.0', tasks={'custom-tool-calling': TaskConfig(type='chat-completion', dataset=DatasetEv(files_url='hf://datasets/xlam-tutorial-ns/xlam-ft-dataset/testing/xlam-test-single.jsonl', id='dataset-3b5YFqDrvUeRBQXurUjUDV', created_at=datetime.datetime(2025, 8, 25, 4, 2, 29, 683838), custom_fields={}, description=None, format=None, hf_endpoint=None, limit=50, name='dataset-3b5YFqDrvUeRBQXurUjUDV', namespace='default', ownership=None, project=None, schema_version='1.0', split=None, type_prefix=None, updated_at=datetime.datetime(2025, 8, 25, 4, 2, 29, 683838), version_id='main', version_tags=[]), metrics={'tool-calling-accuracy': MetricConfig(type='tool-calling', params={'t

In [16]:
# Poll
job = wait_eval_job(evaluator_client, ft_eval_job_id, polling_interval=5, timeout=600)

HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-RFZY5WxsHQ4S6BMQmYb2aA "HTTP/1.1 200 OK"
HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-RFZY5WxsHQ4S6BMQmYb2aA "HTTP/1.1 200 OK"


Job status: running after 5.02 seconds. Progress: 52.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-RFZY5WxsHQ4S6BMQmYb2aA "HTTP/1.1 200 OK"


Job status: running after 10.03 seconds. Progress: 62.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-RFZY5WxsHQ4S6BMQmYb2aA "HTTP/1.1 200 OK"


...

Job status: running after 20.11 seconds. Progress: 92.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-RFZY5WxsHQ4S6BMQmYb2aA "HTTP/1.1 200 OK"


Job status: running after 25.11 seconds. Progress: 100.0%


HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-RFZY5WxsHQ4S6BMQmYb2aA "HTTP/1.1 200 OK"


Job status: completed after 30.12 seconds. Progress: 100%


### 2.2 Review Evaluation Metrics
The following code retrieves the evaluation results for the fine-tuned model evaluation job.

In [17]:
# Get evaluation results for customized model
ft_results = evaluator_client.evaluation.jobs.results(job_id=ft_eval_job_id)
print(ft_results.model_dump_json(indent=2))

HTTP Request: GET http://nemo-evaluator.nemo.svc.cluster.local:7331/v1/evaluation/jobs/eval-RFZY5WxsHQ4S6BMQmYb2aA/results "HTTP/1.1 200 OK"


{
  "job": "eval-RFZY5WxsHQ4S6BMQmYb2aA",
  "id": "evaluation_result-DWuRg5NLvrW5QxPapespoY",
  "created_at": "2025-08-25T04:02:29.847772",
  "custom_fields": {},
  "description": null,
  "files_url": "hf://datasets/evaluation-results/eval-RFZY5WxsHQ4S6BMQmYb2aA",
  "groups": {},
  "namespace": "default",
  "ownership": null,
  "project": null,
  "tasks": {
    "custom-tool-calling": {
      "metrics": {
        "tool-calling-accuracy": {
          "scores": {
            "function_name_accuracy": {
              "value": 0.98,
              "stats": {
                "count": 50,
                "max": null,
                "mean": 0.98,
                "min": null,
                "stddev": null,
                "stderr": null,
                "sum": 49.0,
                "sum_squared": null,
                "variance": null
              }
            },
            "function_name_and_args_accuracy": {
              "value": 0.7,
              "stats": {
                "count": 50,

In [18]:
# Extract function name accuracy score
ft_scores = ft_results.tasks["custom-tool-calling"].metrics["tool-calling-accuracy"].scores
ft_function_name_accuracy_score = ft_scores["function_name_accuracy"].value
ft_function_name_and_args_accuracy = ft_scores["function_name_and_args_accuracy"].value

print(f"Custom model: function_name_accuracy: {ft_function_name_accuracy_score}")
print(f"Custom model: function_name_and_args_accuracy: {ft_function_name_and_args_accuracy}")

Custom model: function_name_accuracy: 0.98
Custom model: function_name_and_args_accuracy: 0.7


A successfully fine-tuned `meta/llama-3.2-1b-instruct` results in a significant increase in tool calling accuracy with 

In this case you should observe roughly the following improvements -
* function_name_accuracy: 12% to 92%
* function_name_and_args_accuracy: 8% to 72%

Since this evaluation was on a limited number of samples for demonstration purposes, you may choose to increase `tasks.dataset.limit` in your evaluation config `simple_tool_calling_eval_config`

## (Optional) Next Steps



* You may also run the same evaluation on a base `meta/llama-3.1-70b-instruct` model for comparison.
For this, first you will need to deploy the corresponding NIM using instructions [here](https://build.nvidia.com/meta/llama-3_1-70b-instruct/deploy). After your NIM is deployed, set that endpoint as your evaluation target like so -

``` python
# Create an evaluation target
NIM_URL = "http://0.0.0.0:8000"
EVAL_TARGET = {
    "type": "model", 
    "model": {
       "api_endpoint": {
         "url": f"{NIM_URL}/v1/completions",
         "model_id": "meta/llama-3.1-70b-instruct",
        }
    }
}

# Start eval job
eval_job = nemo_client.evaluation.jobs.create(
    config=simple_tool_calling_eval_config,
    target=EVAL_TARGET
)
```

Running evaluation using the default config in this notebook, you should observe `meta/llama-3.1-70b-instruct` performance similar to -
* function_name_accuracy: 98%
* function_name_and_args_accuracy: 66%

Remarkably, a LoRA-tuned `meta/llama-3.2-1b-instruct` achieves accuracy that is close to a model 70 times its size, even outperforming it in the combined `function_name_and_args_accuracy` score.

You can now proceed with the same processes to fine-tune other NIM for LLMs and evaluate the accuracies between the base model and the fine-tuned model. By doing so, you can produce more accurate models for your use case.